In [43]:
import argparse
import os
import numpy as np
import pandas as pd
from keras.layers import Conv2D, Input, BatchNormalization, LeakyReLU, ZeroPadding2D, UpSampling2D
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.preprocessing import image as im
import struct
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import json
from PIL import Image

import urllib

In [30]:
    df = pd.read_json(r"C:\Users\ArpitKh4101\Downloads\vehicle-number-plate-detection\Indian_Number_plates.json", lines=True)
    df.head()

,annotation,content,extras
0,"[{'label': ['number_plate'], 'notes': '', 'poi...",http://com.dataturks.a96-i23.open.s3.amazonaws...,NaN
1,"[{'label': ['number_plate'], 'notes': '', 'poi...",http://com.dataturks.a96-i23.open.s3.amazonaws...,NaN
2,"[{'label': ['number_plate'], 'notes': '', 'poi...",http://com.dataturks.a96-i23.open.s3.amazonaws...,NaN
3,"[{'label': ['number_plate'], 'notes': '', 'poi...",http://com.dataturks.a96-i23.open.s3.amazonaws...,NaN
4,"[{'label': ['number_plate'], 'notes': '', 'poi...",http://com.dataturks.a96-i23.open.s3.amazonaws...,NaN


In [47]:
dataset = dict()
dataset["image_name"] = list()
dataset["image_width"] = list()
dataset["image_height"] = list()
dataset["top_x"] = list()
dataset["top_y"] = list()
dataset["bottom_x"] = list()
dataset["bottom_y"] = list()

counter = 0
for index, row in df.iterrows():
    img = urllib.request.urlopen(row["content"])
    img = Image.open(img)
    img = img.convert('RGB')
    img.save(r"C:\Users\ArpitKh4101\Downloads\INDIAN/licensed_car{}.jpeg".format(counter), "JPEG")
    
    dataset["image_name"].append("licensed_car{}".format(counter))
    
    data = row["annotation"]
    
    dataset["image_width"].append(data[0]["imageWidth"])
    dataset["image_height"].append(data[0]["imageHeight"])
    dataset["top_x"].append(data[0]["points"][0]["x"])
    dataset["top_y"].append(data[0]["points"][0]["y"])
    dataset["bottom_x"].append(data[0]["points"][1]["x"])
    dataset["bottom_y"].append(data[0]["points"][1]["y"])
    
    counter += 1
print("Downloaded {} car images.".format(counter))

Downloaded 237 car images.


In [199]:
df = pd.DataFrame(dataset)
df.head()

,image_name,image_width,image_height,top_x,top_y,bottom_x,bottom_y
0,licensed_car0,806,466,0.722084,0.587983,0.868486,0.688841
1,licensed_car1,494,449,0.161943,0.850780,0.582996,1.000000
2,licensed_car2,500,314,0.082000,0.697452,0.230000,0.828025
3,licensed_car3,500,469,0.434000,0.667377,0.718000,0.765458
4,licensed_car4,1001,563,0.208791,0.390764,0.865135,0.984014


In [200]:
df["image_name"] = df["image_name"] + ".jpeg"
df["centre_x"]=(df["top_x"]+df["bottom_x"])/2
df["centre_y"]=(df["top_y"]+df["bottom_y"])/2
df["width"]= -df["top_x"]+ df["bottom_x"]
df["height"]=(-df["top_y"]+df["bottom_y"])
df.drop(["top_x", "top_y","bottom_x","bottom_y","image_width","image_height"], axis=1, inplace=True)
df=df[['centre_x', 'centre_y', 'width', 'height']]
df.head()

,centre_x,centre_y,width,height
0,0.795285,0.638412,0.146402,0.100858
1,0.372470,0.925390,0.421053,0.149220
2,0.156000,0.762739,0.148000,0.130573
3,0.576000,0.716418,0.284000,0.098081
4,0.536963,0.687389,0.656344,0.593250


In [99]:
df.to_csv('NumberPlates.csv')

In [94]:
dg=df
dg.drop(["image_name"],axis=1, inplace=True)
dg.head()

,index,centre_x,centre_y,width,height
0,0,0.795285,0.638412,0.146402,0.100858
1,1,0.372470,0.925390,0.421053,0.149220
2,2,0.156000,0.762739,0.148000,0.130573
3,3,0.576000,0.716418,0.284000,0.098081
4,4,0.536963,0.687389,0.656344,0.593250


In [96]:
np.savetxt(r'Annotations.txt', dg.values)

In [114]:
for i in range(0,237):
    np.savetxt(r"C:\Users\ArpitKh4101\Downloads\Annotations\licensed_car{}.txt".format(i), dg.values[i],delimiter='\t')
    

In [125]:
dat=[]
for i in range(0,237):
    dat.append(r"C:\Users\ArpitKh4101\Downloads\INDIAN\licensed_car{}.jpeg".format(i))

In [126]:
df["Image_add"]=dat

In [127]:
df=df[['Image_add','image_name','centre_x', 'centre_y', 'width', 'height']]

In [128]:
np.savetxt(r"C:\Users\ArpitKh4101\Downloads\Anno1.txt", df.values,delimiter=" ", newline = "\n", fmt="%s")

In [122]:
np.savetxt("data.txt", temp, delimiter=" ", newline = "\n", fmt="%s")

array([['licensed_car0', 0.7952853598014886, 0.6384120171673815,
        0.14640198511166302, 0.10085836909871293],
       ['licensed_car1', 0.37246963562753, 0.9253897550111355,
        0.421052631578948, 0.14922048997772897],
       ['licensed_car2', 0.156, 0.7627388535031845, 0.14800000000000002,
        0.13057324840764306],
       ...,
       ['licensed_car234', 0.481, 0.6882530120481926, 0.294,
        0.159638554216867],
       ['licensed_car235', 0.36860068259385653, 0.6601208459214496,
        0.146757679180887, 0.11782477341389697],
       ['licensed_car236', 0.48481012658227807, 0.49319727891156456,
        0.888607594936708, 0.7278911564625851]], dtype=object)

In [203]:
fd=pd.DataFrame(dataset)
fd.drop(['image_height','image_width'],axis=1, inplace=True)

In [148]:
fd=fd[['image_name','top_x', 'bottom_x','top_y', 'bottom_y']]
np.savetxt(r"C:\Users\ArpitKh4101\Downloads\Anno4.txt", fd.values,delimiter=" ", newline = "\n", fmt="%s")

In [204]:

fd=fd[['image_name','top_x', 'bottom_x','top_y', 'bottom_y']]
fd.head()

,image_name,top_x,bottom_x,top_y,bottom_y
0,licensed_car0,0.722084,0.868486,0.587983,0.688841
1,licensed_car1,0.161943,0.582996,0.850780,1.000000
2,licensed_car2,0.082000,0.230000,0.697452,0.828025
3,licensed_car3,0.434000,0.718000,0.667377,0.765458
4,licensed_car4,0.208791,0.865135,0.390764,0.984014


In [160]:
np.savetxt(r"C:\Users\ArpitKh4101\Downloads\Anno4.txt", fd.values,delimiter=" ", newline = "\n", fmt="%s")

In [205]:
i=0
for x,z in fd.iterrows():
    file1=open(r"C:\Users\ArpitKh4101\Downloads\Annotations1\licensed_car{}.txt".format(i),'w')
    file1.write('0' + " " + str(z[0])+ " "+ str(z[1])+" "+ str(z[2])+" "+str(z[3]))
    i+=1
i

237